# Installing important module

In [22]:
!pip install tensorflow backend

  Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.3 kB)
Using cached tensorflow-2.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (589.8 MB)


# Importing modules

In [23]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

2024-04-09 02:23:11.753861: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 02:23:12.752185: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-09 02:23:14.639895: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


# Installing Kaggle for importing its dataset using the API

In [ ]:
!pip install kaggle

# Creating a Directory and moving kaggle.json file to it

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Downloading the ASL dataset from kaggle using there API command

In [ ]:
!kaggle datasets download -d datamunge/sign-language-mnist

# Extracting the dataset

In [ ]:
from zipfile import ZipFile
dataset = "sign-language-mnist.zip"
with ZipFile(dataset, "r") as zip:
    zip.extractall()
    print("data is extracted")

# Now we can play with the data!

Use Pandas to read CSV files of train and test

In [ ]:
train_df = pd.read_csv("sign_mnist_train.csv")
test_df = pd.read_csv("sign_mnist_test.csv")

Use Head command to view first five rows of the dataset

In [ ]:
train_df.head()

# Data Visualization and Preprocessing

In [ ]:
plt.figure(figsize=(8,8)) # set any size you want
sns.set_style("darkgrid") # we can use white,dark,whitegrid,darkgrid,ticks etc as per our requirements
sns.countplot(data=train_df, x="label")

The dataset seems balanced as for each training label , enough training examples exist

In [ ]:
train_df_only_labels = train_df["label"]
test_df_only_labels =  test_df["label"]
del train_df["label"]
del test_df["label"]

In [ ]:
LabelBinarizer = LabelBinarizer()
y_train = LabelBinarizer.fit_transform(train_df_only_labels) # this convert all the labels in binary form: (0s and 1s)
y_test = LabelBinarizer.fit_transform(test_df_only_labels)

Now I will assign all the values of the dataset except the "labels" to the new variables

In [ ]:
x_train = train_df.values
x_test = test_df.values

Now we will perform a grayscale normalization to reduce the effect of illumination's differences.Moreover the CNN converges faster on [0..1] data than on [0..255].

In [ ]:
# Normalizing the data
x_train = x_train/255
x_test = x_test/255

Reshaping the data from 1-D to 3-D as required through input by CNN's


In [ ]:
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

# Preview of first ten images

In [ ]:
f, ax=plt.subplots(2, 5)
f.set_size_inches(10, 10)
k = 0
for i in range(2):
    for j in range(5):
        ax[i, j].imshow(x_train[k].reshape(28,28), cmap = "gray")
        k+=1
    plt.tight_layout()

# Data Augmentation

Using Data Augmentation to prevent overfitting

In [ ]:
augmented_data = ImageDataGenerator(
    featurewise_center= False,              # set input mean to 0 over the dataset
    samplewise_center= False,               # set each sample mean to 0
    featurewise_std_normalization= False,   # divide input by the std of the dataset
    samplewise_std_normalization= False,    # divide each output by its std
    zca_whitening= False,                   # apply ZCA whitening
    rotation_range= 10,                     # randomly rotate images in the range of (0,180) degree
    zoom_range= 0.1,                        # randomly zoom images
    width_shift_range= 0.1,                 # randomly shift images horizontally (fraction of total width)
    height_shift_range= 0.1,                # randomly shift images vertically (fraction of total height)
    horizontal_flip= False,                 # randomly flip images
    vertical_flip= False,                   # randomly flip images
)
augmented_data.fit(x_train)

For the data augmentation, i choosed to :

Randomly rotate some training images by 10 degrees Randomly Zoom by 10% some training images Randomly shift images horizontally by 10% of the width Randomly shift images vertically by 10% of the height I did not apply a vertical_flip nor horizontal_flip since it could have lead to misclassify.

Once our model is ready, we fit the training dataset .

# CONVOLUTIONAL NEURAL NETWORKS
A Convolutional Neural Network(CNN) is a special type of an Artificial Intelligence implementation which uses a special mathematical matrix manipulation called the convolution operation to process data from the images.

A convolution does this by multiplying two matrices and yielding a third, smaller matrix.
The Network takes an input image, and uses a filter (or kernel) to create a feature map describing the image.
In the convolution operation, we take a filter (usually 2x2 or 3x3 matrix ) and slide it over the image matrix. The coresponding numbers in both matrices are multiplied and and added to yield a single number describing that input space. This process is repeated all over the image.

# Training the Model:

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor="val_accuracy", patience=2, verbose=1, factor=0.5, min_lr=0.00001)

In [ ]:
model = Sequential()
model.add(Conv2D(250, (3,3), strides=1, padding="same", activation="relu", input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2), strides=2, padding="same"))
model.add(Conv2D(225, (3,3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.2))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2), strides=2, padding="same"))
model.add(Conv2D(200, (3,3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(175, (3,3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.3))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(150, (3,3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(125, (3,3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.4))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(100, (3,3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(75, (3,3), strides=1, padding="same", activation="relu"))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(50, (3,3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Conv2D(25, (3,3), strides=1, padding="same", activation="relu"))
model.add(BatchNormalization())
model.add(MaxPool2D((2,2) , strides = 2 , padding = 'same'))
model.add(Flatten())
model.add(Dense(units=512, activation="relu"))
model.add(Dropout(0.6))
model.add(Dense(units=24, activation="relu"))
model.compile(optimizer="adam", loss="categorical_crossentropy", sample_weight_mode=["accuracy"])
model.summary()

In [ ]:
history = model.fit(augmented_data.flow(x_train,y_train, batch_size = 128) ,epochs = 1 , validation_data = (x_test, y_test) , callbacks = [learning_rate_reduction])

In [ ]:
model.evaluate(x_test,y_test)

NameError: name 'model' is not defined